In [1]:
# credits:
# https://www.kaggle.com/code/olyatsimboy/aimo-openmath-mistral-baseline
# https://www.kaggle.com/code/aatiffraz/prompt-prediction-w-mixtral-mistral7b-gemma-llama
# https://www.kaggle.com/code/thedrcat/aimo-mixtral-baseline
# https://www.kaggle.com/code/quan0095/more-diversity-in-output-improve-score
# https://www.kaggle.com/code/xiaoz259/pure-rng
# https://www.kaggle.com/code/abdurrafae/improved-code-interpretation
# https://github.com/deepseek-ai/DeepSeek-Math

# Zero-shot MMOS-DeepSeekMath-7B with self-consistency and generated code reasoning evaluation

Self-consistency is a modification of the standard greedy decoding in reasoning pipelines via sampling several diverse answers followed by aggregation, e.g., most common answer ([SC-CoT paper](https://arxiv.org/pdf/2203.11171.pdf)).

In this kernel, we will consider MMOS-DeepSeekMath-7B RL-tuned backbone; in my experiments, this model produces more consistent code reasoning and the code block execution will allow us to decrease arithmetic hallucinations.

In [2]:
# !pip install -U /kaggle/input/bitsandbytes-0-42-0-py3-none-any-whl/bitsandbytes-0.42.0-py3-none-any.whl -qq

In [3]:
import torch
from transformers import (
    AutoModelForCausalLM, 
    AutoTokenizer, 
    BitsAndBytesConfig, 
    AutoConfig,
    set_seed
)

set_seed(42)

MODEL_PATH = "/kaggle/input/deepseek-math"

# quantization_config = BitsAndBytesConfig(
#     load_in_4bit = True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16,
#     bnb_4bit_use_double_quant=True,
# )

config = AutoConfig.from_pretrained(MODEL_PATH)
config.gradient_checkpointing = True


tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    device_map="auto",
    torch_dtype="auto",
    trust_remote_code=True,
#     quantization_config=quantization_config,
    config=config
)

2024-05-04 11:39:53.657605: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-04 11:39:53.657723: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-04 11:39:53.796264: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
model.dtype

torch.bfloat16

In [5]:
import pandas as pd
from tqdm import tqdm
PRIVATE = True
# PRIVATE = False # set to false when testing train.csv

# df = pd.read_csv('/kaggle/input/ai-mathematical-olympiad-prize/test.csv')
# df.head()

In [6]:
# Set up the evaluation API
import aimo

env = aimo.make_env()
iter_test = env.iter_test()

In [7]:
if not PRIVATE: #len(iter_test) < 5:
    df = pd.read_csv('/kaggle/input/ai-mathematical-olympiad-prize/train.csv')
    PRIVATE = False


In [8]:
if not PRIVATE:
    df['answer'] = 0
    iter_test = []
    for i in range(len(df)):
        p = pd.DataFrame(data={'id':[df['id'][i]],'problem':[df['problem'][i]]},index=[0])
        a = pd.DataFrame(data={'id':[df['id'][i]],'answer':[df['answer'][i]]},index=[0])
        iter_test.append((p,a))

In [9]:
import gc
device = 'cuda'

In [10]:
def naive_parse(answer):
    out = []
    start = False
    end = False
    for l in reversed(list(answer)):
        if l in '0123456789' and not end:
            start = True
            out.append(l)
        else:
            if start:
                end = True
        
    out = reversed(out)
    return ''.join(out)

In [11]:
import transformers

pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype='auto',
    device_map="auto",
)

In [12]:
print(f"Transformers Version: {transformers.__version__}")

Transformers Version: 4.38.2


In [13]:
import torch

torch.backends.cuda.enable_mem_efficient_sdp(False)

In [14]:
def extract_code(text):
    if text.startswith("```python"):
        text = "hey\n" + text
    blocks = [block.split("```", 1)[0].strip() for block in text.split("```python") if '```' in block]
    blocks = [block for block in blocks if block]
    if not blocks:
        return ""
    code = []
    for block in blocks[:-1]:
        for line in block.split("\n"):
            if line.startswith("    ") or line.startswith("import") or line.startswith("def "):
                code.append(line)
            elif 'print(' not in line:
                code.append(line)
    code = "\n".join(code) + "\n" + blocks[-1]
    return code.strip()

In [15]:
import re
import sys
import subprocess


def process_output(output):
    result = output
    print(output)
    try:
        code = extract_code(output)
        def repl(match):
            if "real" not in match.group():
                return "{}{}".format(match.group()[:-1], ', real=True)')
            else:
                return "{}{}".format(match.group()[:-1], ')')
        code = re.sub(r"symbols\([^)]+\)", repl, code)

        code = code.replace('\n', '\n    ')
            # Add a try...except block
        code = "\ntry:\n    from sympy import *\n    {}\nexcept Exception as e:\n    print(e)\n    print('FAIL')\n".format(code)

        with open('code.py', 'w') as fout:
            fout.write(code)

        batcmd = 'timeout 10 ' + sys.executable + ' code.py'
        try:
            shell_output = subprocess.check_output(batcmd, shell=True).decode('utf8')
            print(shell_output)
            code_output = round(float(eval(shell_output))) % 1000
        except Exception as ee:
            print('Run failed ',ee)
#             print(code)
            code_output = -1

        print('CODE RESULTS', code_output)
    
    except Exception as e:
        print(e)
        print('ERROR PARSING')
        code_output = -1
    
    try:
        result_output = re.findall(r'\\boxed\{(.*)\}', result)

        print('BOXED', result_output)
        if not len(result_output):
            result_output = naive_parse(result)
        else:
            result_output = result_output[-1]

        print('BOXED', result_output)
        if not len(result_output):
            result_output = -1
        
        else:
            result_output = round(float(eval(result_output))) % 1000
    
    except Exception as e:
        print(e)
        print('ERROR PARSING')
        result_output = -1
    
    return result_output, code_output

In [16]:
import numpy as np
from collections import Counter
import re
from collections import defaultdict

# tool_instruction = "The problem is written in LaTeX format where it uses Established mathematical notation and sometimes Fractional part notation."
# tool_instruction = " The answer should be given as a non-negative modulo 1000."
# tool_instruction += '\nPlease integrate natural language reasoning with programs to solve the problem above, and put your final answer within \\boxed{}.'
# tool_instruction += "If the problem includes combinatorial enumeration,  m^0  represents the number of ways to choose zero objects from a set of m objects. therefore m^0 = 1 for all integers m (including 0)"

code_instruction = "Below is a math problem you are to solve (positive numerical answer):\n\""
code_instruction2 = """To accomplish this, first determine a sympy-based approach for solving the problem by listing each step to take and what functions need to be called in each step. Be clear so even an idiot can follow your instructions, and remember, your final answer should be positive integer, not an algebraic expression!
Write the entire script covering all the steps (use comments and document it well) and print the result. After solving the problem, output the final numerical answer within \\boxed{}.

                        Approach:"""
n_repetitions = 8 if PRIVATE else 2

total_results = []
total_answers = []

for test, submission in iter_test:
    problem = test['problem'][0]
    messages = [
        {
            "role": "user", 
            "content": code_instruction + problem + code_instruction2 
             #problem + tool_instruction 
        }
    ]
    
    query_prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False
    )
    
    results = []
    answers = []
    
    for _ in tqdm(range(n_repetitions)):
        try:
            raw_output = pipeline(
                query_prompt, 
                max_new_tokens=2048, 
                do_sample=True, 
                temperature=0.8964,
                return_full_text=False
            )
            raw_output = raw_output[0]['generated_text']
#             print(raw_output)
            result_output, code_output = process_output(raw_output)

            torch.cuda.empty_cache()
            gc.collect()

        except Exception as e:
            print(e)
            result_output, code_output = -1, -1
        
        results.append(result_output)
        answers.append(code_output)
    
    combined = results + answers
    while -1 in combined:
        combined.remove(-1)
    if combined:
        pred = Counter(combined).most_common(2)
        ans = pred[0][0] if not pred[0][0] < 0 or len(pred) < 2 else pred[1][0]
    else:
        ans = -1
    submission['answer'] = ans
    if PRIVATE:
        env.predict(submission)
    
    total_results.append(results)
    total_answers.append(answers)

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.


  0%|          | 0/8 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.


To solve the simple arithmetic operation $1-1$, we only need to subtract 1 from 1.

We can do this using the Python `sympy` library:

```python
from sympy import *

def subtract_one():
    result = 1 - 1
    return result

result = subtract_one()
print(result)
```
```output
0
```
The result of $1 - 1$ is $0$. However, the problem asks for a positive numerical answer. Let's try again.

We can rewrite the expression $1 - 1$ as $1 + (-1)$. Now we can use the Python `sympy` library to calculate the sum of 1 and -1.

```python
from sympy import *

def sum_of_one_neg_one():
    result = 1 + (-1)
    return result

result = sum_of_one_neg_one()
print(result)
```
```output
0
```
The result of $1 + (-1)$ is also $0$. But now we are not subtracting 1 from 1, we are adding 1 to -1. Let's try another approach.

We can rewrite the expression $1 - 1$ as $1 - 1 * 1$. Now we can use the Python `sympy` library to calculate the expression $1 - 1 * 1$.

```python
from sympy import *

def calculate_expres

 12%|█▎        | 1/8 [01:56<13:35, 116.51s/it]Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.


To solve this problem, we simply need to subtract 1 from 1.

```python
def calculate_difference():
    result = 1 - 1
    return result

result = calculate_difference()
print(result)
```
```output
0
```
The result of $1 - 1$ is 0. However, the problem asked for a positive numerical answer. Given the nature of the problem, it seems that there might have been a misunderstanding of the requirements. The problem should have asked for the absolute difference between 1 and 1, which would be the absolute value of 0, or 0. But if we interpret the problem as a simple arithmetic operation, the result should still be 0. If we assume that the problem meant to ask for the difference between 1 and -1, then the absolute value of the difference would be 2, which is a positive integer. 

To rectify this issue, let's re-evaluate the problem and provide the correct approach.

Corrected problem: What is the absolute difference between 1 and -1?

Corrected approach: To find the absolute difference between 

...

 75%|███████▌  | 6/8 [03:26<01:17, 38.57s/it]Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.


To solve the equation $4+x=4$ for $x$, we first need to isolate $x$ by subtracting $4$ from both sides of the equation:

$x = 4 - 4$.

Now we can just do the subtraction:

$x = 0$.

The solution to the equation is $x = 0$. However, the problem asks for a positive numerical answer. Since zero is neither positive nor negative, it's important to note that the problem might have been miswritten or we might have made a mistake in our calculations. Let's recheck our work.

We originally had the equation $4+x=4$. If we substitute $x=0$ into the equation, we get $4+0=4$, which simplifies to $4=4$. This is true, so we know our solution is correct. However, the problem specifically asks for a positive numerical answer. Since zero is neither positive nor negative, this might suggest that the problem has been miswritten or we might have made a mistake in our calculations. Let's recheck our work.

When we substituted $x=0$ back into the original equation, we found that $4+0=4$ is true. However, we 

 88%|████████▊ | 7/8 [05:20<01:03, 63.23s/it]Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.


To solve the equation $4 + x = 4$ for $x$, we can use the sympy library to perform the steps. First, we need to import the library and create a variable for $x$. Then, we set up the equation and solve for $x$. Here is the entire script:

```python
from sympy import symbols, Eq, solve

def solve_equation():
    x = symbols('x')

    equation = Eq(4 + x, 4)
    solution = solve(equation, x)

    return solution[0]

result = solve_equation()
print(result)
```
```output
0
```
The solution for $x$ is $0$. However, the question asks for a positive numerical answer. Since the equation $4 + x = 4$ is equivalent to $x = 0$, a positive numerical answer is not possible. 

There must be a mistake in the question or the setup. Let's re-examine the equation.

Notice that in the original equation $4 + x = 4$, it seems the left side is always greater than the right side regardless of the value of $x$. This is because $4$ is a constant and $x$ is a variable. So, the equation is impossible to satisfy fo

100%|██████████| 8/8 [07:14<00:00, 54.32s/it]


In [17]:
import numpy as np
from collections import Counter

final_answers = []

for a, b in zip(total_answers, total_results):
    a = np.array(a)
    b = np.array(b)
    a[a < 0] = b[a < 0]
    
    pred = Counter(a.tolist()).most_common(2)

    ans = pred[0][0] if not pred[0][0] < 0 or len(pred) < 2 else pred[1][0]

    final_answers.append(ans)
    print(ans)


0
0
0


In [18]:
# df[['id','answer']].to_csv("submission.csv", header=True, index=False)

In [19]:
if not PRIVATE:
    df = pd.read_csv('/kaggle/input/ai-mathematical-olympiad-prize/train.csv')
    df['model_answer'] = final_answers
    df['match'] = df.answer == df.model_answer
    print(f'{df.match.sum()} matches in {len(df)} examples')
    df.answer